'''bye is short for Byte Pair Encoder. It Translate arbitrary utf-8 string into sequences of integers, where each integer represents small chunks of commonly occuring charaters'''

In [1]:
import os
import regex as re
import json
import requests

import torch

'''
Every possible byte (really an integer 0.255) gets mapped by openAI to a unicode character that represents it visually some bytes have
their apperarence preserved because they dont cause any trouble for eg chr(33) return "!" so in the return dictorny we have d[0] -> 'A'
instead which is just chr(0 + 2**8)

In [2]:
#def get_encoder():
home_dir = 'home/princeton/Downloads/Project/'
cache_dir = os.path.join(home_dir, 'mingpt')
os.makedirs(cache_dir, exist_ok=True)
print(cache_dir)

home/princeton/Downloads/Project/mingpt


In [3]:
encoder_local_file = 'encoder.json'
#print(encoder_local_file)
#encoder_remote_file = 'https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json'
#print(encoder_remote_file)
#get_file(encoder_local_file, encoder_remote_file)
with open(encoder_local_file, 'r') as f:
    encoder = json.load(f)
assert len(encoder) == 50257

In [4]:
vocab_local_file = 'vocab.bpe'
vocab_remote_file = 'https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe'
with open(vocab_local_file, 'r', encoding='utf-8') as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
assert len(bpe_merges) == 50000

In [5]:
#def bytes_to_unicode():
    # the 188 integers that render fine in their original form and need no shifting
bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
print(bs)
print("-----------------------")
print(len(bs))
cs = bs[:] # all integers b in bs will simply map to chr(b) in the output dict
print("-----")
print(cs)
print(len(cs))
n = 0
for b in range(2**8):
      if b not in bs:
            # if this byte is "ugly" then map it to the next available "nice" character
          bs.append(b)
          cs.append(2**8+n)
          n += 1
print("\n","-------")
print(bs)
print(len(bs))
print("-----")
print(cs)
print(len(cs))
cs = [chr(n) for n in bs]

print(cs)
print("----")
d = dict(zip(bs,cs))
print(d)

[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]
-----------------------
188
-----
[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,

In [6]:
cache = {}
byte_encoder = d
byte_decoder = {v:k for k,v in byte_encoder.items()}
print(byte_decoder)

{'!': 33, '"': 34, '#': 35, '$': 36, '%': 37, '&': 38, "'": 39, '(': 40, ')': 41, '*': 42, '+': 43, ',': 44, '-': 45, '.': 46, '/': 47, '0': 48, '1': 49, '2': 50, '3': 51, '4': 52, '5': 53, '6': 54, '7': 55, '8': 56, '9': 57, ':': 58, ';': 59, '<': 60, '=': 61, '>': 62, '?': 63, '@': 64, 'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72, 'I': 73, 'J': 74, 'K': 75, 'L': 76, 'M': 77, 'N': 78, 'O': 79, 'P': 80, 'Q': 81, 'R': 82, 'S': 83, 'T': 84, 'U': 85, 'V': 86, 'W': 87, 'X': 88, 'Y': 89, 'Z': 90, '[': 91, '\\': 92, ']': 93, '^': 94, '_': 95, '`': 96, 'a': 97, 'b': 98, 'c': 99, 'd': 100, 'e': 101, 'f': 102, 'g': 103, 'h': 104, 'i': 105, 'j': 106, 'k': 107, 'l': 108, 'm': 109, 'n': 110, 'o': 111, 'p': 112, 'q': 113, 'r': 114, 's': 115, 't': 116, 'u': 117, 'v': 118, 'w': 119, 'x': 120, 'y': 121, 'z': 122, '{': 123, '|': 124, '}': 125, '~': 126, '¡': 161, '¢': 162, '£': 163, '¤': 164, '¥': 165, '¦': 166, '§': 167, '¨': 168, '©': 169, 'ª': 170, '«': 171, '¬': 172, '®': 

In [7]:
decoder = {v:k for k,v in encoder.items()}
print(decoder)

{0: '!', 1: '"', 2: '#', 3: '$', 4: '%', 5: '&', 6: "'", 7: '(', 8: ')', 9: '*', 10: '+', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '<', 28: '=', 29: '>', 30: '?', 31: '@', 32: 'A', 33: 'B', 34: 'C', 35: 'D', 36: 'E', 37: 'F', 38: 'G', 39: 'H', 40: 'I', 41: 'J', 42: 'K', 43: 'L', 44: 'M', 45: 'N', 46: 'O', 47: 'P', 48: 'Q', 49: 'R', 50: 'S', 51: 'T', 52: 'U', 53: 'V', 54: 'W', 55: 'X', 56: 'Y', 57: 'Z', 58: '[', 59: '\\', 60: ']', 61: '^', 62: '_', 63: '`', 64: 'a', 65: 'b', 66: 'c', 67: 'd', 68: 'e', 69: 'f', 70: 'g', 71: 'h', 72: 'i', 73: 'j', 74: 'k', 75: 'l', 76: 'm', 77: 'n', 78: 'o', 79: 'p', 80: 'q', 81: 'r', 82: 's', 83: 't', 84: 'u', 85: 'v', 86: 'w', 87: 'x', 88: 'y', 89: 'z', 90: '{', 91: '|', 92: '}', 93: '~', 94: '¡', 95: '¢', 96: '£', 97: '¤', 98: '¥', 99: '¦', 100: '§', 101: '¨', 102: '©', 103: 'ª', 104: '«', 105: '¬', 106: '®', 107: '¯', 108: '°', 109: '±', 110: '²'

In [9]:
bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
print(bpe_ranks)

{('Ġ', 't'): 0, ('Ġ', 'a'): 1, ('h', 'e'): 2, ('i', 'n'): 3, ('r', 'e'): 4, ('o', 'n'): 5, ('Ġt', 'he'): 6, ('e', 'r'): 7, ('Ġ', 's'): 8, ('a', 't'): 9, ('Ġ', 'w'): 10, ('Ġ', 'o'): 11, ('e', 'n'): 12, ('Ġ', 'c'): 13, ('i', 't'): 14, ('i', 's'): 15, ('a', 'n'): 16, ('o', 'r'): 17, ('e', 's'): 18, ('Ġ', 'b'): 19, ('e', 'd'): 20, ('Ġ', 'f'): 21, ('in', 'g'): 22, ('Ġ', 'p'): 23, ('o', 'u'): 24, ('Ġa', 'n'): 25, ('a', 'l'): 26, ('a', 'r'): 27, ('Ġt', 'o'): 28, ('Ġ', 'm'): 29, ('Ġo', 'f'): 30, ('Ġ', 'in'): 31, ('Ġ', 'd'): 32, ('Ġ', 'h'): 33, ('Ġan', 'd'): 34, ('i', 'c'): 35, ('a', 's'): 36, ('l', 'e'): 37, ('Ġt', 'h'): 38, ('i', 'on'): 39, ('o', 'm'): 40, ('l', 'l'): 41, ('en', 't'): 42, ('Ġ', 'n'): 43, ('Ġ', 'l'): 44, ('s', 't'): 45, ('Ġ', 're'): 46, ('v', 'e'): 47, ('Ġ', 'e'): 48, ('r', 'o'): 49, ('l', 'y'): 50, ('Ġb', 'e'): 51, ('Ġ', 'g'): 52, ('Ġ', 'T'): 53, ('c', 't'): 54, ('Ġ', 'S'): 55, ('i', 'd'): 56, ('o', 't'): 57, ('Ġ', 'I'): 58, ('u', 't'): 59, ('e', 't'): 60, ('Ġ', 'A'): 61, ('Ġ

In [10]:
text = "Hello this is your last day to live"
bpe_idx = []
pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
tokens = re.findall(pat,text)
print(tokens)

['Hello', ' this', ' is', ' your', ' last', ' day', ' to', ' live']


In [15]:
#bpe
token1 = ''
for token in tokens:
    token_bytes = token.encode('utf-8')
    #print(token_bytes)
    token_translated = ''.join(byte_encoder[b] for b in token_bytes)
    token1+=token_translated
print(token1)

Hello this is your last day to live


In [16]:
word = token1
pairs = set()
prev_char = word[0]
for char in word[1:]:
    pairs.add((prev_char, char))
    prev_char = char
print(pairs)

{('l', 'l'), ('o', 'u'), ('i', 's'), ('v', 'e'), ('a', 'y'), (' ', 'y'), ('s', 't'), ('r', ' '), (' ', 'l'), ('e', 'l'), ('s', ' '), ('t', 'o'), ('u', 'r'), ('l', 'a'), ('l', 'o'), ('t', ' '), ('y', 'o'), ('i', 'v'), (' ', 't'), ('h', 'i'), ('a', 's'), ('y', ' '), ('t', 'h'), ('o', ' '), (' ', 'd'), ('l', 'i'), ('d', 'a'), ('H', 'e'), (' ', 'i')}


In [17]:
#function for bpe
if token1 in cache:
    print(cache[token])
word = tuple(token1)
pairs = pairs
if not pairs:
    print(token)

while True:
    bigram = min(pairs, key=lambda pair: bpe_ranks.get(pair, float('inf')))
    if bigram not in bpe_ranks:
        break
    first, second = bigram

    new_word = []
    i = 0
    while i < len(word):
        try:
            j = word.index(first, i)
            new_word.extend(word[i:j])
            i = j
        except:
            new_word.extend(word[i:])
            break

        if word[i] == first and i < len(word)-1 and word[i+1] == second:
            new_word.append(first+second)
            i += 2
        else:
            new_word.append(word[i])
            i += 1

    new_word = tuple(new_word)
    word = new_word
    if len(word) == 1:
        break
    else:
        pairs 

word = ' '.join(word)
cache[token1] = word
print(word)
        


KeyboardInterrupt: 

In [20]:
assert isinstance(text, str)
idx = encoder.encode(text)


AttributeError: 'dict' object has no attribute 'encode'